In [2]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from pathlib import Path
import ot
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from utils import LoadCloudPoint, DistanceProfile
from utils import plot_3d_points_and_connections
from utils import compute_W_matrix_distance_matrix_input
from accuracy import accuracy
import numpy as np
import heapq

In [3]:
import random

random.seed(10)

lcp = LoadCloudPoint(filepath="datasets/csv_files/0005_Jogging001.csv")
source_pc, target_pc = lcp.get_two_random_point_cloud()

dp = DistanceProfile(source_pc, target_pc)
distance_matrix = dp.compute_L2_matrix()

Loaded point cloud data from datasets\csv_files\0005_Jogging001.csv, number of frames: 1377


In [4]:
def knn_geodesic_distances(points, k):
    """
    Compute geodesic (shortest-path) distances over the kNN graph for a 3D point cloud.

    Parameters:
        points (np.ndarray): shape (N, 3) point cloud
        k (int): number of nearest neighbors

    Returns:
        np.ndarray: shape (N, N) geodesic distance matrix
    """
    points = np.asarray(points)
    N = points.shape[0]

    # ==== Build full Euclidean distance matrix ====
    diff = points[:, None, :] - points[None, :, :]
    dist_matrix = np.sqrt(np.sum(diff**2, axis=2))

    # ==== Determine k nearest neighbors for each point ====
    knn_indices = np.argsort(dist_matrix, axis=1)[:, 1:k+1]  # skip self (index 0)

    # ==== Build adjacency list for the kNN graph ====
    graph = [[] for _ in range(N)]
    for i in range(N):
        for j in knn_indices[i]:
            w = dist_matrix[i, j]
            graph[i].append((j, w))
            graph[j].append((i, w))  # make graph symmetric

    # ==== Dijkstra's algorithm for a single source ====
    def dijkstra(start):
        dist = np.full(N, np.inf)
        dist[start] = 0.0
        pq = [(0.0, start)]

        while pq:
            current_dist, u = heapq.heappop(pq)
            if current_dist > dist[u]:
                continue

            for v, w in graph[u]:
                new_dist = current_dist + w
                if new_dist < dist[v]:
                    dist[v] = new_dist
                    heapq.heappush(pq, (new_dist, v))

        return dist

    # ==== Compute all-pairs geodesic distances ====
    geodesic_matrix = np.vstack([dijkstra(i) for i in range(N)])

    return geodesic_matrix


In [28]:
distance_matrix = []
distance_matrix.append(knn_geodesic_distances(source_pc, k=3))
distance_matrix.append(knn_geodesic_distances(target_pc, k=3))


W, map_matrix= compute_W_matrix_distance_matrix_input(distance_matrix[0], distance_matrix[1])
plot_3d_points_and_connections(source_pc, target_pc, map_matrix)

In [29]:

accuracy(map_matrix)

0.6153846153846154

In [26]:
dp = DistanceProfile(source_pc, target_pc)
distance_matrix = dp.compute_L2_matrix()

W, map_matrix= compute_W_matrix_distance_matrix_input(distance_matrix[0], distance_matrix[1])
plot_3d_points_and_connections(source_pc, target_pc, map_matrix)

In [27]:
accuracy(map_matrix)

0.8846153846153846

In [20]:
l2_accuracies = []
knn_accuracies = []
k_int_list = []

for _ in range(100):
    flag = False
    k_int = 1
    lcp = LoadCloudPoint(filepath="datasets/csv_files/0005_Jogging001.csv")
    source_pc, target_pc = lcp.get_two_random_point_cloud()

    dp = DistanceProfile(source_pc, target_pc)
    distance_matrix = dp.compute_L2_matrix()

    while not flag:
        distance_knn_matrix = dp.compute_knn_geodesic_distance_matrix(k=k_int)
        if np.isinf(distance_knn_matrix[0]).any() or np.isinf(distance_knn_matrix[1]).any():
            k_int += 1
        else:
            flag = True

    k_int_list.append(k_int)
    W, map_matrix= compute_W_matrix_distance_matrix_input(distance_matrix[0], distance_matrix[1])
    W2, map_matrix2= compute_W_matrix_distance_matrix_input(distance_knn_matrix[0], distance_knn_matrix[1])

    print("L2 accuracy:", accuracy(map_matrix))
    print("kNN Geodesic accuracy:", accuracy(map_matrix2))

    l2_accuracies.append(accuracy(map_matrix))
    knn_accuracies.append(accuracy(map_matrix2))

print("Average L2 accuracy over 5 runs:", np.mean(l2_accuracies))
print("Average kNN Geodesic accuracy over 5 runs:", np.mean(knn_accuracies))




Loaded point cloud data from datasets\csv_files\0005_Jogging001.csv, number of frames: 1377
L2 accuracy: 0.34615384615384615
kNN Geodesic accuracy: 0.38461538461538464
Loaded point cloud data from datasets\csv_files\0005_Jogging001.csv, number of frames: 1377
L2 accuracy: 0.4230769230769231
kNN Geodesic accuracy: 0.34615384615384615
Loaded point cloud data from datasets\csv_files\0005_Jogging001.csv, number of frames: 1377
L2 accuracy: 0.46153846153846156
kNN Geodesic accuracy: 0.4230769230769231
Loaded point cloud data from datasets\csv_files\0005_Jogging001.csv, number of frames: 1377
L2 accuracy: 0.4230769230769231
kNN Geodesic accuracy: 0.38461538461538464
Loaded point cloud data from datasets\csv_files\0005_Jogging001.csv, number of frames: 1377
L2 accuracy: 0.5
kNN Geodesic accuracy: 0.7692307692307693
Loaded point cloud data from datasets\csv_files\0005_Jogging001.csv, number of frames: 1377
L2 accuracy: 0.15384615384615385
kNN Geodesic accuracy: 0.07692307692307693
Loaded point

In [21]:
k_int_list

[3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 5,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 5,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 5,
 3,
 4,
 3,
 4,
 3,
 3,
 3]